In [2]:
import csv
import re

input_file = 'processed_prescales_2017.csv'
output_file = 'processed_prescales_2017_filtered.csv'

pattern = re.compile(r'DST_HT410_PFScouting_v\d+/0')

with open(input_file, 'r') as infile, open(output_file, 'w', newline='') as outfile:
    reader = csv.reader(infile)
    writer = csv.writer(outfile)
    
    for row in reader:
        if len(row) > 4 and not pattern.search(row[4]):
            writer.writerow(row)

print(f"Filtered data has been written to {output_file}")

Filtered data has been written to processed_prescales_2017_filtered.csv


In [10]:
import csv
from collections import defaultdict

def process_file(filename):
    runs = defaultdict(list)
    consistent_runs = set()

    with open(filename, 'r') as f:
        reader = csv.reader(f)
        for row_num, row in enumerate(reader, 1):
            if not row:
                print(f"Warning: Empty row at line {row_num}")
                continue
            
            try:
                run_number = row[0]
                l1_htt280er_value = next((col.split('/')[1] for col in row if col.startswith('L1_HTT280er')), None)
                if l1_htt280er_value is None:
                    print(f"Warning: L1_HTT280er not found in row {row_num}")
                    continue
                runs[run_number].append(l1_htt280er_value)
            except IndexError:
                print(f"Error: Unexpected row format at line {row_num}: {row}")
                continue

    for run, values in runs.items():
        if all(value == '1.00' for value in values):
            consistent_runs.add(run)

    return sorted(consistent_runs)

filename = 'processed_prescales_2017_filtered.csv'
consistent_runs = process_file(filename)

print("Runs with L1_HTT280er consistently set to 1.00:")
print(consistent_runs)

Runs with L1_HTT280er consistently set to 1.00:
['301046', '301141', '301142', '301161', '301165', '301179', '301180', '301183', '301281', '301283', '301298', '301323', '301330', '301359', '301384', '301391', '301392', '301393', '301396', '301397', '301398', '301399', '301417', '301447', '301448', '301449', '301450', '301461', '301472', '301475', '301476', '301519', '301524', '301529', '301530', '301531', '301532', '301567', '301627', '301664', '301665', '301694', '301912', '301913', '301914', '301941', '301959', '301960', '301970', '301984', '301985', '301986', '301987', '301997', '301998', '302019', '302026', '302029', '302031', '302033', '302034', '302037', '302038', '302040', '302041', '302042', '302043', '302131', '302159', '302163', '302165', '302166', '302225', '302228', '302229', '302240', '302262', '302263', '302277', '302279', '302280', '302322', '302328', '302337', '302342', '302343', '302344', '302350', '302388', '302392', '302393', '302448', '302472', '302473', '302474', '

In [13]:
import csv
from collections import defaultdict

L1_TRIGGERS = [
    'L1_SingleJet180', 'L1_SingleJet200', 'L1_HTT200er', 'L1_HTT240er',
    'L1_HTT270er', 'L1_HTT280er', 'L1_HTT300er', 'L1_HTT320er',
    'L1_HTT380er', 'L1_HTT400er', 'L1_HTT450er', 'L1_HTT500er'
]

def process_file(filename):
    runs = defaultdict(lambda: defaultdict(list))
    consistent_runs = defaultdict(set)

    with open(filename, 'r') as f:
        reader = csv.reader(f)
        for row_num, row in enumerate(reader, 1):
            if not row:
                print(f"Warning: Empty row at line {row_num}")
                continue
            
            try:
                run_number = int(row[0])  # Convert to integer
                for trigger in L1_TRIGGERS:
                    value = next((col.split('/')[1] for col in row if col.startswith(trigger)), None)
                    if value is None:
                        print(f"Warning: {trigger} not found in row {row_num}")
                    else:
                        runs[run_number][trigger].append(value)
            except (IndexError, ValueError):
                print(f"Error: Unexpected row format at line {row_num}: {row}")
                continue

    for run, triggers in runs.items():
        for trigger, values in triggers.items():
            if all(value == '1.00' for value in values):
                consistent_runs[trigger].add(run)

    return {trigger: sorted(list(runs)) for trigger, runs in consistent_runs.items()}

filename = 'processed_prescales_2017_filtered.csv'
l1_trigger_runs = process_file(filename)

# Print results
for trigger, runs in l1_trigger_runs.items():
    print(f"{trigger}: {runs}")

L1_SingleJet180: [300087, 300105, 300106, 300107, 300117, 300122, 300123, 300155, 300156, 300157, 300226, 300233, 300234, 300235, 300236, 300237, 300238, 300239, 300240, 300280, 300281, 300282, 300283, 300284, 300364, 300372, 300375, 300389, 300399, 300400, 300401, 300459, 300461, 300462, 300463, 300464, 300466, 300467, 300497, 300514, 300515, 300516, 300517, 300558, 300560, 300574, 300575, 300576, 300631, 300632, 300633, 300635, 300636, 300673, 300674, 300675, 300676, 300742, 300777, 300780, 300785, 300806, 300811, 300812, 300816, 300817, 301046, 301141, 301142, 301161, 301165, 301179, 301180, 301183, 301281, 301283, 301298, 301323, 301330, 301359, 301384, 301391, 301392, 301393, 301396, 301397, 301398, 301399, 301417, 301447, 301448, 301449, 301450, 301461, 301472, 301475, 301476, 301519, 301524, 301529, 301530, 301531, 301532, 301567, 301627, 301664, 301665, 301694, 301912, 301913, 301914, 301941, 301959, 301960, 301970, 301984, 301985, 301986, 301987, 301997, 301998, 302019, 302026